In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import itertools
import matplotlib.patches
from scipy.stats import pearsonr



In [2]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')


# Create group column based on case_type to simplify group names
metadata['group'] = metadata['case_type'].map({
    'case-lesional skin': 'skin-ADL',
    'case-nonlesional skin': 'skin-ADNL', 
    'control-nonlesional skin': 'skin-H',
    'case-anterior nares': 'nares-AD',
    'control-anterior nares': 'nares-H'
})

metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,sex,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,male,4/16/2015,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,female,8/11/2015,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,female,11/20/2014,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,female,9/23/2015,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL


In [3]:
# Read in table at ASV level
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/feature_table_with_tax_labels_Genus.biom'
biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus.biom'
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus-ASV-non-collapse.biom'
biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the index
df.index = df.index.str.replace('15564.', '')

# Get samples that don't start with 'skin' in metadata
skin_samples = metadata[metadata['group'].str.startswith('skin')].index

# Filter df to keep only skin samples
df = df.loc[skin_samples]
df

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Eubacterium_M,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,65.0,7.0,170.0,0,5.0,20.0,0,14.0,0,19.0,...,0,0,0,0,0,0,0,0,0,0
900221,0,27.0,0,0,2.0,3.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca010EBL,22.0,132.0,65.0,0,19.0,31.0,24.0,0,0,6.0,...,0,0,0,0,0,0,0,0,0,0
900460,150.0,32.0,50.0,3.0,0,2.0,28.0,4.0,0,13.0,...,0,0,0,0,0,0,0,0,0,0
900051,33.0,222.0,24.0,0,10.0,0,0,0,0,13.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004032,78.0,15.0,2.0,0,102.0,0,28.0,0,62.0,2.0,...,0,0,0,0,0,0,0,0,0,0
Ca006ONL,19.0,135.0,59.0,1.0,14.0,16.0,0,0,0,29.0,...,0,0,0,0,0,0,0,0,0,0
Ca006ONL2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def rclr_transform(df, pseudocount=1e-6):
    """
    Applies Robust Centered Log-Ratio (RCLR) transformation to a DataFrame.
    Zeros are ignored in the geometric mean calculation per sample.
    """
    # Convert sparse to dense if needed
    df = df.sparse.to_dense()

    # Replace 0 with np.nan to ignore in log and mean
    df_masked = df.replace(0, np.nan)

    # Apply log (with pseudocount only where needed)
    log_df = np.log(df_masked + pseudocount)

    # Subtract mean of each row (feature)
    rclr_df = log_df.sub(log_df.mean(axis=1, skipna=True), axis=0)

    return rclr_df



rclr_df = rclr_transform(df)

In [5]:
# Map the 'group' column from metadata to df based on matching index
rclr_df['group'] = metadata.loc[rclr_df.index, 'group']
rclr_df

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1.744767,-0.483710,2.706178,NaN,-0.820182,0.566112,NaN,0.209437,NaN,0.514819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900221,NaN,1.126537,NaN,NaN,-1.476152,-1.070687,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca010EBL,1.055786,2.847545,2.139130,NaN,0.909182,1.398730,1.142797,NaN,NaN,-0.243497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900460,3.510481,1.965581,2.411868,-0.401542,NaN,-0.807007,1.832050,-0.113860,NaN,1.064795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900051,1.435857,3.342027,1.117404,NaN,0.241935,NaN,NaN,NaN,NaN,0.504299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004032,2.337271,0.688612,-1.326291,NaN,2.605535,NaN,1.312766,NaN,2.107696,-1.326291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca006ONL,1.128272,3.089107,2.261370,-1.816166,0.822890,0.956421,NaN,NaN,NaN,1.551128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca006ONL2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL


In [6]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()
        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df

In [7]:
# Filter for skin samples
skin_samples = rclr_df[rclr_df['group'].isin(['skin-ADL', 'skin-ADNL', 'skin-H'])]
skin_samples

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1.744767,-0.483710,2.706178,NaN,-0.820182,0.566112,NaN,0.209437,NaN,0.514819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900221,NaN,1.126537,NaN,NaN,-1.476152,-1.070687,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca010EBL,1.055786,2.847545,2.139130,NaN,0.909182,1.398730,1.142797,NaN,NaN,-0.243497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900460,3.510481,1.965581,2.411868,-0.401542,NaN,-0.807007,1.832050,-0.113860,NaN,1.064795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
900051,1.435857,3.342027,1.117404,NaN,0.241935,NaN,NaN,NaN,NaN,0.504299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004032,2.337271,0.688612,-1.326291,NaN,2.605535,NaN,1.312766,NaN,2.107696,-1.326291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca006ONL,1.128272,3.089107,2.261370,-1.816166,0.822890,0.956421,NaN,NaN,NaN,1.551128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL
Ca006ONL2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL


In [8]:
# Map o_scorad and pid from metadata to skin_samples based on matching indexes
skin_samples = skin_samples.merge(metadata[['o_scorad']], left_index=True, right_index=True)
skin_samples = skin_samples.merge(metadata[['pid']], left_index=True, right_index=True)
skin_samples = skin_samples.merge(metadata[['area']], left_index=True, right_index=True)

skin_samples

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group,o_scorad,pid,area
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1.744767,-0.483710,2.706178,NaN,-0.820182,0.566112,NaN,0.209437,NaN,0.514819,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,40,Ca-009-ST,Cape Town
900221,NaN,1.126537,NaN,NaN,-1.476152,-1.070687,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,34,Ca-101-ID,Umtata
Ca010EBL,1.055786,2.847545,2.139130,NaN,0.909182,1.398730,1.142797,NaN,NaN,-0.243497,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,21,Ca-010-EB,Cape Town
900460,3.510481,1.965581,2.411868,-0.401542,NaN,-0.807007,1.832050,-0.113860,NaN,1.064795,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,40,Ca-146-SM,Umtata
900051,1.435857,3.342027,1.117404,NaN,0.241935,NaN,NaN,NaN,NaN,0.504299,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,41,Ca-011-LQ,Cape Town
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004032,2.337271,0.688612,-1.326291,NaN,2.605535,NaN,1.312766,NaN,2.107696,-1.326291,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,78,Ca-126-KB,Umtata
Ca006ONL,1.128272,3.089107,2.261370,-1.816166,0.822890,0.956421,NaN,NaN,NaN,1.551128,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,34,Ca-006-ON,Cape Town
Ca006ONL2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,34,Ca-006-ON,Cape Town


In [9]:
# Convert 'o_scorad' to numeric (coerce errors to NaN)
skin_samples['o_scorad'] = pd.to_numeric(metadata['o_scorad'], errors='coerce')

# Drop rows with missing SCORAD values
skin_samples = skin_samples.dropna(subset=['o_scorad'])

# Filter to only high severity samples
# skin_samples = skin_samples[skin_samples['o_scorad'] >= 25]

# Subset to only lesional samples
skin_samples = skin_samples[skin_samples['group'] == 'skin-ADL']

skin_samples


,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group,o_scorad,pid,area
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1.744767,-0.483710,2.706178,NaN,-0.820182,0.566112,NaN,0.209437,NaN,0.514819,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,40.0,Ca-009-ST,Cape Town
900221,NaN,1.126537,NaN,NaN,-1.476152,-1.070687,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,34.0,Ca-101-ID,Umtata
Ca010EBL,1.055786,2.847545,2.139130,NaN,0.909182,1.398730,1.142797,NaN,NaN,-0.243497,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,21.0,Ca-010-EB,Cape Town
900460,3.510481,1.965581,2.411868,-0.401542,NaN,-0.807007,1.832050,-0.113860,NaN,1.064795,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,40.0,Ca-146-SM,Umtata
900051,1.435857,3.342027,1.117404,NaN,0.241935,NaN,NaN,NaN,NaN,0.504299,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,41.0,Ca-011-LQ,Cape Town
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004002,2.208196,3.189025,2.043893,-0.189699,1.419739,-1.288311,1.602060,0.097983,NaN,1.014274,...,NaN,NaN,-1.288311,NaN,NaN,NaN,skin-ADL,38.0,Ca-124-AB,Umtata
900403,2.789983,3.905690,1.987636,-0.171848,-0.171848,0.338978,0.926765,1.368597,NaN,0.808982,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,78.0,Ca-126-KB,Umtata
9004032,2.337271,0.688612,-1.326291,NaN,2.605535,NaN,1.312766,NaN,2.107696,-1.326291,...,NaN,NaN,NaN,NaN,NaN,NaN,skin-ADL,78.0,Ca-126-KB,Umtata


In [10]:
taxa_list = [' g__Streptococcus', ' g__Staphylococcus', ' g__Micrococcus', ' g__Veillonella_A']


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

colors = {'Cape Town': 'salmon', 'Umtata': 'tan'}

# Set square dimensions per subplot
# square_size = 3  # in inches
n_cols = len(taxa_list)
n_rows = 1

fig, axes = plt.subplots(
    n_rows,
    n_cols,
    figsize=(7, 4),
    sharey=True
)

# Ensure axes is always iterable (even if n_cols == 1)
if n_cols == 1:
    axes = [axes]

for i, taxon in enumerate(taxa_list):
    ax = axes[i]
    
    # Drop missing values
    df = skin_samples[['o_scorad', taxon, 'area']].dropna()

    # Plot colored scatter manually
    for area, group in df.groupby('area'):
        ax.scatter(
            group['o_scorad'],
            group[taxon],
            color=colors.get(area, 'gray'),
            label=area if i == 0 else "",
            alpha=0.5,
            s=20
        )

    # Regression line
    sns.regplot(
        data=df,
        x='o_scorad',
        y=taxon,
        scatter=False,
        line_kws={'color': 'black'},
        ax=ax
    )

    # Pearson correlation
    r, pval = pearsonr(df['o_scorad'], df[taxon])
    r_label = f"Pearson r = {r:.2f}\np = {pval:.1e}"
    ax.text(0.05, 0.95, r_label, transform=ax.transAxes,
            fontsize=10, va='top', ha='left',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    # Axis formatting
    ax.set_title(taxon.replace('g__', ''), fontsize=10)
    ax.set_xlabel("SCORAD", fontsize=12)
    if i == 0:
        ax.set_ylabel("RCLR-transformed abundance", fontsize=12)
    else:
        ax.set_ylabel("")
        ax.set_yticklabels([])

    ax.set_ylim(-3, 7)

# Get handles and labels from one of the subplots
handles, labels = axes[0].get_legend_handles_labels()

# Add a single horizontal legend across the bottom of the figure
fig.legend(
    handles,
    labels,
    loc='lower center',
    ncol=len(labels),
    fontsize=10,
    title_fontsize=11,
    frameon=True,
    bbox_to_anchor=(0.5, 0.025)  # Adjust spacing below the figure
)


plt.tight_layout(rect=[0, 0, 1, 0.92], w_pad=1)

plt.suptitle("Differential Bacteria on Skin and AD Severity", fontsize=14, y=0.98)
plt.subplots_adjust(bottom=0.25)
plt.savefig('../Plots/Analysis_figures/Severity_Correlations/rclr_abundance_vs_severity_ASV_skin.png', dpi=600)


# Nares

In [12]:
# Read in table at ASV level
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/feature_table_with_tax_labels_Genus.biom'
biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus.biom'
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus-ASV-non-collapse.biom'
biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the index
df.index = df.index.str.replace('15564.', '')

# Get samples that don't start with 'nares' in metadata
nares_samples = metadata[metadata['group'].str.startswith('nares')].index

# Filter df to keep only nares samples
df = df.loc[nares_samples]
df

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Eubacterium_M,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,51.0,0,32.0,0,193.0,0,0,0,48.0,0,...,0,0,0,0,0,0,0,0,0,0
900223,3.0,0,0,0,23.0,0,0,0,2.0,0,...,0,0,0,0,0,0,0,0,0,0
Ca010EBPN,83.0,4.0,100.0,0,55.0,0,8.0,0,23.0,29.0,...,0,0,0,0,0,0,0,0,0,0
900462,169.0,2.0,36.0,36.0,73.0,0,7.0,4.0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
900053,188.0,0,0,144.0,0,0,0,0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,31.0,5.0,10.0,0,148.0,0,5.0,0,60.0,4.0,...,0,0,0,0,0,0,0,0,0,0
900402,21.0,74.0,20.0,0,119.0,21.0,0,14.0,0,7.0,...,0,0,0,0,0,0,0,0,0,0
9004022,9.0,0,14.0,301.0,11.0,0,4.0,0,6.0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
rclr_df = rclr_transform(df)
rclr_df

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Eubacterium_M,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,1.302853,NaN,0.836764,NaN,2.633718,NaN,NaN,NaN,1.242229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900223,-1.194695,NaN,NaN,NaN,0.842187,NaN,NaN,NaN,-1.600160,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ca010EBPN,1.786005,-1.246541,1.972335,NaN,1.374498,NaN,-0.553394,NaN,0.502659,0.734460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900462,3.839613,-0.597138,2.293233,2.293233,3.000174,NaN,0.655625,0.096009,-0.191673,-1.290285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900053,4.044725,NaN,NaN,3.778096,NaN,NaN,NaN,NaN,-1.191716,-1.191716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.011916,0.187367,0.880514,NaN,3.575142,NaN,0.187367,NaN,2.672274,-0.035776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900402,1.352545,2.612087,1.303754,NaN,3.087146,1.352545,NaN,0.947079,NaN,0.253932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9004022,0.252154,NaN,0.693986,3.762039,0.452824,NaN,-0.558776,NaN,-0.153311,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Map the 'group' column from metadata to df based on matching index
rclr_df['group'] = metadata.loc[rclr_df.index, 'group']
rclr_df

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,1.302853,NaN,0.836764,NaN,2.633718,NaN,NaN,NaN,1.242229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900223,-1.194695,NaN,NaN,NaN,0.842187,NaN,NaN,NaN,-1.600160,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,1.786005,-1.246541,1.972335,NaN,1.374498,NaN,-0.553394,NaN,0.502659,0.734460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900462,3.839613,-0.597138,2.293233,2.293233,3.000174,NaN,0.655625,0.096009,-0.191673,-1.290285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,4.044725,NaN,NaN,3.778096,NaN,NaN,NaN,NaN,-1.191716,-1.191716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.011916,0.187367,0.880514,NaN,3.575142,NaN,0.187367,NaN,2.672274,-0.035776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900402,1.352545,2.612087,1.303754,NaN,3.087146,1.352545,NaN,0.947079,NaN,0.253932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
9004022,0.252154,NaN,0.693986,3.762039,0.452824,NaN,-0.558776,NaN,-0.153311,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [15]:
# Filter for nares samples
nares_samples = rclr_df[rclr_df['group'].isin(['nares-AD', 'nares-H'])]
nares_samples

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Lachnospira,g__Helcococcus,g__Enterenecus,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,1.302853,NaN,0.836764,NaN,2.633718,NaN,NaN,NaN,1.242229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900223,-1.194695,NaN,NaN,NaN,0.842187,NaN,NaN,NaN,-1.600160,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,1.786005,-1.246541,1.972335,NaN,1.374498,NaN,-0.553394,NaN,0.502659,0.734460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900462,3.839613,-0.597138,2.293233,2.293233,3.000174,NaN,0.655625,0.096009,-0.191673,-1.290285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,4.044725,NaN,NaN,3.778096,NaN,NaN,NaN,NaN,-1.191716,-1.191716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.011916,0.187367,0.880514,NaN,3.575142,NaN,0.187367,NaN,2.672274,-0.035776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900402,1.352545,2.612087,1.303754,NaN,3.087146,1.352545,NaN,0.947079,NaN,0.253932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
9004022,0.252154,NaN,0.693986,3.762039,0.452824,NaN,-0.558776,NaN,-0.153311,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [16]:
# Map o_scorad and pid from metadata to skin_samples based on matching indexes
nares_samples = nares_samples.merge(metadata[['o_scorad']], left_index=True, right_index=True)
nares_samples = nares_samples.merge(metadata[['pid']], left_index=True, right_index=True)
nares_samples = nares_samples.merge(metadata[['area']], left_index=True, right_index=True)

nares_samples

,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group,o_scorad,pid,area
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,1.302853,NaN,0.836764,NaN,2.633718,NaN,NaN,NaN,1.242229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40,Ca-009-ST,Cape Town
900223,-1.194695,NaN,NaN,NaN,0.842187,NaN,NaN,NaN,-1.600160,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34,Ca-101-ID,Umtata
Ca010EBPN,1.786005,-1.246541,1.972335,NaN,1.374498,NaN,-0.553394,NaN,0.502659,0.734460,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21,Ca-010-EB,Cape Town
900462,3.839613,-0.597138,2.293233,2.293233,3.000174,NaN,0.655625,0.096009,-0.191673,-1.290285,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40,Ca-146-SM,Umtata
900053,4.044725,NaN,NaN,3.778096,NaN,NaN,NaN,NaN,-1.191716,-1.191716,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41,Ca-011-LQ,Cape Town
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.011916,0.187367,0.880514,NaN,3.575142,NaN,0.187367,NaN,2.672274,-0.035776,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54,Ca-125-IM,Umtata
900402,1.352545,2.612087,1.303754,NaN,3.087146,1.352545,NaN,0.947079,NaN,0.253932,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,NaN,Ca-124-AB,Umtata
9004022,0.252154,NaN,0.693986,3.762039,0.452824,NaN,-0.558776,NaN,-0.153311,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38,Ca-124-AB,Umtata


In [17]:
# Convert 'o_scorad' to numeric (coerce errors to NaN)
nares_samples['o_scorad'] = pd.to_numeric(metadata['o_scorad'], errors='coerce')

# Drop rows with missing SCORAD values
nares_samples = nares_samples.dropna(subset=['o_scorad'])

# Filter to only high severity samples
# skin_samples = skin_samples[skin_samples['o_scorad'] >= 25]

# Subset to only lesional samples
nares_samples = nares_samples[nares_samples['group'] == 'nares-AD']

nares_samples


,g__Streptococcus,g__Staphylococcus,g__,g__Haemophilus_D_734546,g__Corynebacterium,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Cutibacterium,...,g__Evtepia,g__Fimenecus,g__CAG-41,g__Selenomonas_B_42762,g__Facklamia_A_322620,g__Eubacterium_G,group,o_scorad,pid,area
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,1.302853,NaN,0.836764,NaN,2.633718,NaN,NaN,NaN,1.242229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0,Ca-009-ST,Cape Town
900223,-1.194695,NaN,NaN,NaN,0.842187,NaN,NaN,NaN,-1.600160,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34.0,Ca-101-ID,Umtata
Ca010EBPN,1.786005,-1.246541,1.972335,NaN,1.374498,NaN,-0.553394,NaN,0.502659,0.734460,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21.0,Ca-010-EB,Cape Town
900462,3.839613,-0.597138,2.293233,2.293233,3.000174,NaN,0.655625,0.096009,-0.191673,-1.290285,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0,Ca-146-SM,Umtata
900053,4.044725,NaN,NaN,3.778096,NaN,NaN,NaN,NaN,-1.191716,-1.191716,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41.0,Ca-011-LQ,Cape Town
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900399,-1.158252,3.368956,1.582588,NaN,0.546496,0.345825,NaN,-1.158252,NaN,2.425267,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0,Ca-125-IM,Umtata
9003992,2.011916,0.187367,0.880514,NaN,3.575142,NaN,0.187367,NaN,2.672274,-0.035776,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0,Ca-125-IM,Umtata
9004022,0.252154,NaN,0.693986,3.762039,0.452824,NaN,-0.558776,NaN,-0.153311,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38.0,Ca-124-AB,Umtata


In [18]:
taxa_list = [' g__Streptococcus', ' g__Staphylococcus', ' g__Micrococcus', ' g__Veillonella_A']


In [19]:
colors = {'Cape Town': 'salmon', 'Umtata': 'tan'}

# Set square dimensions per subplot
# square_size = 3  # in inches
n_cols = len(taxa_list)
n_rows = 1

fig, axes = plt.subplots(
    n_rows,
    n_cols,
    figsize=(7, 4),
    sharey=True
)

# Ensure axes is always iterable (even if n_cols == 1)
if n_cols == 1:
    axes = [axes]

for i, taxon in enumerate(taxa_list):
    ax = axes[i]
    
    # Drop missing values
    df = nares_samples[['o_scorad', taxon, 'area']].dropna()

    # Plot colored scatter manually
    for area, group in df.groupby('area'):
        ax.scatter(
            group['o_scorad'],
            group[taxon],
            color=colors.get(area, 'gray'),
            label=area if i == 0 else "",
            alpha=0.5,
            s=20
        )

    # Regression line
    sns.regplot(
        data=df,
        x='o_scorad',
        y=taxon,
        scatter=False,
        line_kws={'color': 'black'},
        ax=ax
    )

    # Pearson correlation
    r, pval = pearsonr(df['o_scorad'], df[taxon])
    r_label = f"Pearson r = {r:.2f}\np = {pval:.1e}"
    ax.text(0.05, 0.95, r_label, transform=ax.transAxes,
            fontsize=10, va='top', ha='left',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    # Axis formatting
    ax.set_title(taxon.replace('g__', ''), fontsize=10)
    ax.set_xlabel("SCORAD", fontsize=12)
    if i == 0:
        ax.set_ylabel("RCLR-transformed abundance", fontsize=12)
    else:
        ax.set_ylabel("")
        ax.set_yticklabels([])

    ax.set_ylim(-3, 7)

# Get handles and labels from one of the subplots
handles, labels = axes[0].get_legend_handles_labels()

# Add a single horizontal legend across the bottom of the figure
fig.legend(
    handles,
    labels,
    loc='lower center',
    ncol=len(labels),
    fontsize=10,
    title_fontsize=11,
    frameon=True,
    bbox_to_anchor=(0.5, 0.025)  # Adjust spacing below the figure
)


plt.tight_layout(rect=[0, 0, 1, 0.92], w_pad=1)

plt.suptitle("Differential Bacteria on Nares and AD Severity", fontsize=14, y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.subplots_adjust(wspace=0.2)  # Add this to increase spacing

plt.savefig('../Plots/Analysis_figures/Severity_Correlations/rclr_abundance_vs_severity_ASV_nares.png', dpi=600)
